<a href="https://colab.research.google.com/github/fmigas/Projects/blob/main/Source_code_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The objective is to recognize a source code programming language. There are projects in as much as 19 different programming languages in a dataset.

The task can be treated as an NLP classification task, while keeping in mind that a source code is not a natural language in terms of syntax, not to mention semantics.
But we can use the same toolbox.

There are four main attitudes that could be applied:
1. treating the source code as a bag of words and using some text classification tools, like sklearn multiclass algorithms fitted with tfidf vectorized text
2. feature engineering  - building a set of new variables out of the text, like length, number of different types of brackets, ratio of upper/lower characters in a code etc.
and then building a classifier on top of these features
3. treating the source as a sequence with some positional dependencies that could be learned by a recurrent deep learning model (like bidirectional LSTM);
in this scenario we must also decide whether to apply a char-based or token-based attitude. Probably it would not work in a classification of a natural language text task, but taking into consideration how much more structured and rigid is a programming language code syntax, and how much more limited is its vocabulary, it could be worth trying.
LSTM networks are costly to train, but our train dataset is rather small, so why not give it a try. Some positional dependencies, like "import ... as ..." in Python or for loops structure in C/C++ may be very distinctive.
4. And last but not least - we can search for a ready-made tool (e.g. library) that was developed for this specific task.
Sometimes reinventing the wheel is a waste of time and energy.
I found a library code_tokenize (https://github.com/cedricrupb/code_tokenize) which seems to do this job very well.
Unfortunately, it has only a limited number of programming languages implemented and some of the languages from our dataset are not on the list.
However, looking for a proven, ready made solution is always an option.

A mixed attitude could be a mix of the above methods.
Having both BOW preprocessed and vectorized (with CountVectorizer, TFIDF or anything else) and new features, we can try two paths:
1. build two separate base models and build an ensemble model on top of them (there are many options to choose from), or
2. build a deep learning model (in Keras or Pytorch) with two input streams of data, concatenated inside the model

One thing that we'll skip in this task is avoiding a data leakage effect. According to the best practices, we should conduct some processing (e.g. tfidf vectorization) with a fit_transform method only on a train dataset and later on transform on a test set, but for the sake of simplicity, we'll let it go this time.

The last technical comment. We will never use a lower() function on any processed text for this task. Upper / lower cases in programming languages have their meaning so by lowering all cases we would lose quite a lot of information.

Let's start.

In [ ]:
import pandas as pd
import json

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Python_data/data.csv')

# verifying null data
df.isna().sum()

language     0
proj_id      0
file_id      0
file_body    4
dtype: int64

In [ ]:
# the only thing we can do with 4 rows with null data in a 'file_body' is to drop them
df.dropna(inplace = True)
df.reset_index(inplace = True)

I will only run only a very basic EDA here.



In [ ]:
df.head(10)

,index,language,proj_id,file_id,file_body
0,0,JavaScript,10001,100001,// Functions as values of a variable\nvar cube...
1,1,JavaScript,10001,100002,// Functions as values of a variable\nvar cube...
2,2,JavaScript,10002,100003,function median(ary) {\n if (ary.length == ...
3,3,JavaScript,10002,100004,"[\n null,\n 4,\n 3.5,\n 2.1\n]\n"
4,4,JavaScript,10002,100005,(() => {\n 'use strict';\n\n // median :...
5,5,JavaScript,10003,100006,"function divByZero(dividend,divisor)\n{\n\tvar..."
6,6,JavaScript,10004,100007,"if (""abs"" in Math) { ... }\n"
7,7,JavaScript,10004,100008,"if (typeof bloop !== ""undefined"") { ... }\n"
8,8,JavaScript,10005,100009,(function () {\n 'use strict';\n\n // di...
9,9,JavaScript,10005,100010,"function sort_disjoint(values, indices) {\n v..."


We can easily see some projects have more than one file.
As our task is to label projects (not files) it would be useful to concatenate source codes from different files belonging to a certain project into one string. We'll do it soon.

In [ ]:
proj = df['proj_id'].value_counts()
print(proj)

17404    41
19276    19
10888    15
11158    14
11364    14
         ..
15238     1
15237     1
15236     1
15235     1
20009     1
Name: proj_id, Length: 10006, dtype: int64


As we can see, some projects are composed of many files (max. 41), some are composed of 1 file.


Now let's see how many unique labels we have.

In [ ]:
langs = df['language'].value_counts()
print(langs)
print(f"\nThere are {len(langs)} different languages in our dataset")

Python         1452
Haskell        1333
Perl           1135
Ruby           1104
JavaScript     1104
C              1078
Go              998
Java            987
Mathematica     926
C++             880
Scala           802
Fortran         745
Kotlin          643
Julia           637
R               603
MATLAB          544
PHP             477
Rust            417
Swift           403
Name: language, dtype: int64

There are 19 different languages in our dataset


There are as many as 19 different source code languages, quite a lot, so we have a multilabel classification ahead of us. Moreover, the dataset seems to be unballanced.

So let's concatenate source code from different files belonging to a certain project in one. We will join them with a new line symbol.

In [ ]:
df = df.groupby(['proj_id', 'language'], as_index = False).agg({'file_body': "\n".join})

In [ ]:
df.head(10)

,proj_id,language,file_body
0,10001,JavaScript,// Functions as values of a variable\nvar cube...
1,10002,JavaScript,function median(ary) {\n if (ary.length == ...
2,10003,JavaScript,"function divByZero(dividend,divisor)\n{\n\tvar..."
3,10004,JavaScript,"if (""abs"" in Math) { ... }\n\nif (typeof bloop..."
4,10005,JavaScript,(function () {\n 'use strict';\n\n // di...
5,10006,JavaScript,"var Таблица_замен = [\n [ 4, 10, 9, 2, 13, ..."
6,10007,JavaScript,function shellSort (a) {\n for (var h = a.l...
7,10008,JavaScript,"[[1, 3, 5], [2, 4, 6]]\n\n(function () {\n ..."
8,10009,JavaScript,(function () {\n\n // hanoi :: Int -> Strin...
9,10010,JavaScript,(function () {\n 'use strict';\n\n // Ja...


In [ ]:
df.shape

(10006, 3)

We have 10006 rows - the same number as for unique 'proj_id' values, so we have it confirmed that all proj_id / file_id pairs were labelled with the same programming language. That seems logical, but it's always good to do such a sanity check.

In [ ]:
df.to_pickle('/content/drive/MyDrive/Python_data/data_concatenated.pkl')

In this notebook, I will test two very different methods:



1.   Classification with a TFIDF vectorization method on a Bag-of-Words with a couple of basic algorithms like logistic regression, Naive Bayes etc.
1.   Classification with an LSTM bidirectional deep learning model on char-based code split



**TFIDF ON BAG-OF-WORDS**

The first decision we need to make is what tokenizer to use. I guess the results could be obtained with a dedicated tokenizer with custom regex rules, but for the purpose of this task I found NLTK WordPunctTokenizer good enough.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

from nltk.tokenize import WordPunctTokenizer
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import LinearSVC

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/Python_data/data_concatenated.pkl')

In [ ]:
df.head()

,proj_id,language,file_body
0,10001,JavaScript,// Functions as values of a variable\nvar cube...
1,10002,JavaScript,function median(ary) {\n if (ary.length == ...
2,10003,JavaScript,"function divByZero(dividend,divisor)\n{\n\tvar..."
3,10004,JavaScript,"if (""abs"" in Math) { ... }\n\nif (typeof bloop..."
4,10005,JavaScript,(function () {\n 'use strict';\n\n // di...


In [ ]:
tokenizer = WordPunctTokenizer()

df['tokens'] = df['file_body'].apply(tokenizer.tokenize)
df['tokens'] = df['tokens'].apply(" ".join) # tokenizer returns a list, we need to convert this back to a string separated with white spaces

In [ ]:
df.head()

,proj_id,language,file_body,tokens
0,10001,JavaScript,// Functions as values of a variable\nvar cube...,// Functions as values of a variable var cube ...
1,10002,JavaScript,function median(ary) {\n if (ary.length == ...,function median ( ary ) { if ( ary . length ==...
2,10003,JavaScript,"function divByZero(dividend,divisor)\n{\n\tvar...","function divByZero ( dividend , divisor ) { va..."
3,10004,JavaScript,"if (""abs"" in Math) { ... }\n\nif (typeof bloop...","if ("" abs "" in Math ) { ... } if ( typeof bloo..."
4,10005,JavaScript,(function () {\n 'use strict';\n\n // di...,( function () { ' use strict '; // disjointSor...


In [ ]:
encoder = LabelEncoder() # all algorithms used in this section require labelled independent features, not one-hote-encoded
y = encoder.fit_transform(df['language'])

In [ ]:
vectorizer = TfidfVectorizer(lowercase = False, tokenizer=str.split, max_features = 3000) # we enforce using a very simple tokenizer based on white space split
# max_features = 3000 is an arbitrary choice, we could try different values
X = vectorizer.fit_transform(df['tokens'])

We choose a couple of very different models to see which works the best.

In [ ]:
models = [
    LogisticRegression(multi_class='multinomial', solver='lbfgs'),
    RandomForestClassifier(),
    BernoulliNB(),
    LinearSVC(),
]

We try each model on a stratified kfold split.

In [ ]:
results = []
for model in models:
    model_results = {}
    cv = StratifiedKFold(n_splits=10, random_state=1, shuffle = True)
    try:
        n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    except:
        n_scores = cross_val_score(model, X.toarray(), y, scoring='accuracy', cv=cv, n_jobs=-1) # some models need a sparse matrix as an input
    model_results['name'] = model
    model_results['acc'] = np.mean(n_scores)

    print(f"Mean Accuracy for {model}: {np.mean(n_scores):.3f} at std dev {np.std(n_scores):.3f}")
    results.append(model_results)

res = pd.DataFrame(results).sort_values(['acc'], ascending = False)
res.set_index('name', inplace = True)
print(res)

Mean Accuracy for LogisticRegression(multi_class='multinomial'): 0.952 at std dev 0.005
Mean Accuracy for RandomForestClassifier(): 0.960 at std dev 0.005
Mean Accuracy for BernoulliNB(): 0.883 at std dev 0.011
Mean Accuracy for LinearSVC(): 0.960 at std dev 0.004
                                                    acc
name                                                   
RandomForestClassifier()                       0.960024
LinearSVC()                                    0.959825
LogisticRegression(multi_class='multinomial')  0.951730
BernoulliNB()                                  0.883072


We reached 96% accuracy on a strongly unballanced dataset with 19 classes, which seems to be a decent result. We could dig deeper and check quite a lot of other options, like:


*   Deep learning model in tensorflow/pytorch
*   Optimizing an SVC model with different kernels etc.
*   Good result for a RandomForrestClassifier suggests we could get even better with boosted models like XGBoost. Usually optimizing hypermarameters with a package like Optuna improves the model quality even more.

Another obvious step for a real-world scenario is to analyze the 4% of cases not recognized correctly to search for an inspiration for improvement.

I also checked if feature selection with a chi2 method adds value to the model, but (I was a bit surprised) it depreciated the result by 1-2pp.

Now let's try a completely different method - a recurrent model on char-based sequences.

**LSTM MODEL**

As we will aply a char-based attitude, the further steps in data processing are very different in comparison to a Bag-of-words application.

At first, we need to build a set of all chars used in our source codes and investigate it.

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/Python_data/data_concatenated.pkl')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10006 entries, 0 to 10005
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   proj_id    10006 non-null  int64 
 1   language   10006 non-null  object
 2   file_body  10006 non-null  object
dtypes: int64(1), object(2)
memory usage: 234.6+ KB


We need to build a set of all chars used in the code texts.

In [ ]:
vocab = list(df['file_body'])
vocab = " ".join(vocab)
len(vocab)
chars = set(vocab)
chars = list(chars)
print(chars)
print(len(chars))
print(len(vocab))

['æ', '%', '¥', '١', '^', '̂', '⊞', 'C', 'ù', 'Ó', 'ב', 'h', '⣔', '¼', '𠀀', 'N', '∫', '⢗', '𝔦', '⠁', '६', ']', 'è', 'f', '♜', '⁷', 'U', '`', '∂', '♥', '⌘', '┳', 'ְ', '令', '3', 'क', '我', 'í', 'ë', '©', '¤', '豬', '₥', '\t', '╟', 'Ñ', 'O', 'λ', '♖', '好', 'Ĵ', 'ſ', '₃', '♝', '间', '३', '⃝', 'э', '╗', '٤', 'þ', '本', 'व', '九', '4', 'Ⅻ', 'ê', '北', '♦', '्', 'ज', 'Е', 'H', 'ç', 'о', 'Â', '∅', '₴', 'ï', 'o', '7', '₄', '┌', 'ƒ', 'ŧ', 'ι', '┛', 'ế', 'A', '₦', '}', 'Ë', 'ĥ', 'ь', '♫', '⁵', '∉', 'п', '馬', 'Ł', '┘', 'ÿ', '二', 'あ', '〚', '☻', '↓', '☹', '⣉', '┐', '&', 'Ô', 'ш', 'Î', 'ο', 'ø', 'k', '२', '₭', '鸡', 'т', 'τ', 'ю', 'π', '⣁', '鼠', '∖', 'S', '♛', 'ß', '→', '羊', '微', 'з', 'Σ', '\\', 'स', '∆', 'ǎ', '╔', '️', 'Ö', '₨', 'É', '~', 'र', 'φ', 'Ш', '久', '田', '点', 'Я', 'ő', 'Ĳ', 'õ', '𝔫', '╶', '\xa0', 'О', '±', 'प', '–', '⡹', '¡', '≈', 'д', '┴', '今', '║', '₮', '⠶', 'ч', 'Æ', '说', 'ä', 'ﬆ', '└', '˗', '┏', '₢', '٦', '·', 'с', '人', '.', 'м', '┫', '७', 'T', '\n', '≥', 'а', '[', '🐫', '٨', '下', 'œ', '♡', '9'

There are 647 different chars in an over 11 milion char long string, some of them look a bit strange at first glance. Let's investigate. 

In [ ]:
# We build dictionary of all chars with their count on a full source code column.
chars_count = {s: vocab.count(s) for s in chars}
chars_count = dict(sorted(chars_count.items(), key = lambda x: x[1]))
print(chars_count)

{'١': 1, '̂': 1, '⣔': 1, '¼': 1, '𠀀': 1, '∫': 1, '⢗': 1, '⠁': 1, '♜': 1, '⁷': 1, '∂': 1, '┳': 1, 'ְ': 1, '令': 1, '豬': 1, '╟': 1, '♝': 1, '间': 1, '╗': 1, '٤': 1, '本': 1, 'Ⅻ': 1, 'Е': 1, 'ƒ': 1, 'ŧ': 1, '┛': 1, 'ế': 1, 'ĥ': 1, '♫': 1, '⁵': 1, '∉': 1, 'п': 1, '馬': 1, '〚': 1, '↓': 1, '☹': 1, 'ш': 1, 'ο': 1, '鸡': 1, 'τ': 1, '⣁': 1, '鼠': 1, '∖': 1, '羊': 1, 'ǎ': 1, '╔': 1, 'Ш': 1, '田': 1, '点': 1, 'Я': 1, 'ő': 1, '⡹': 1, '¡': 1, '┴': 1, '说': 1, 'ﬆ': 1, '˗': 1, '┏': 1, '٦': 1, '┫': 1, '🐫': 1, '٨': 1, '下': 1, 'œ': 1, '♡': 1, 'け': 1, 'П': 1, 'ﬁ': 1, '与': 1, 'ǒ': 1, 'К': 1, '╚': 1, '⢰': 1, '┓': 1, '🐖': 1, '⠔': 1, '╮': 1, '地': 1, '⡏': 1, '頰': 1, 'ŉ': 1, 'ж': 1, 'ﬀ': 1, '╹': 1, '٧': 1, 'ﬂ': 1, 'С': 1, '🐜': 1, 'ū': 1, '╝': 1, '語': 1, '🐑': 1, 'Þ': 1, '글': 1, '╋': 1, '🐗': 1, '⠝': 1, '딸': 1, '한': 1, '٢': 1, '╺': 1, '不': 1, '母': 1, 'ω': 1, 'त': 1, '╸': 1, '╯': 1, 'ή': 1, 'Ж': 1, '╰': 1, '肖': 1, 'Ǆ': 1, '🌲': 1, '🐚': 1, '٣': 1, '┻': 1, '🐛': 1, 'ǌ': 1, '〇': 1, 'Ε': 1, '猴': 1, '™': 1, '↑': 1, '⢱': 1, '〛': 1,

We see that plenty of chars are very rarely used - like once or ten times or fewer than 100 out of 11 milion chars totally.

I assumed we can make a cut at 1000 occurances of a char, starting from a tilde ~ char. Let's build a dictionary with which we'll get rid of extremely rare chars from the code. Our assumption is that getting rid of them will simplify the model at a very limited or no cost for its quality.

In [ ]:
chars_dict = {s: c for s, c in chars_count.items() if c > 1000}
print(chars_dict)

{'~': 1363, '`': 2078, '^': 2174, 'Z': 2475, 'J': 3041, 'Q': 3182, 'K': 4235, 'Y': 5136, 'X': 5358, '?': 5446, 'V': 6033, '@': 6924, 'H': 9950, 'U': 10360, 'G': 10734, 'W': 10739, '&': 12087, '|': 13108, '\\': 14895, 'q': 15042, '%': 15599, '9': 15877, '7': 16038, '8': 16264, 'z': 16612, 'j': 17319, '!': 17741, 'B': 18418, 'F': 18744, 'O': 20683, '6': 20889, 'M': 21107, 'P': 22655, '#': 22734, 'D': 23231, 'R': 24879, '4': 25952, '5': 26028, 'L': 26894, 'N': 26912, 'C': 27505, 'A': 31410, '3': 32498, 'E': 32628, '$': 33197, 'k': 35118, 'T': 37426, 'I': 37743, '*': 41076, '<': 41096, 'S': 43035, '+': 44282, "'": 44373, '>': 48889, '/': 56176, '_': 56259, '}': 57557, '{': 57572, 'w': 59270, 'v': 59455, '2': 59477, ']': 62212, '[': 62402, 'x': 65434, 'y': 74276, ':': 79519, 'b': 85360, '\t': 86422, ';': 96665, '0': 99413, '1': 101716, '-': 102072, 'g': 106710, 'h': 118992, '"': 121227, '.': 137123, '=': 140698, 'f': 147609, 'm': 157851, 'p': 161713, 'u': 174199, 'c': 188113, 'd': 189579, '

Now let's clean the code and get rid of rare chars.
We will also save the data file for an lstm model and the chars dictionary.

In [ ]:
def clean_code(code):
    return [x for x in code if x in chars_dict.keys()]

df['clean'] = df['file_body'].apply(clean_code)

df.drop(['file_body'], inplace = True, axis = 1) # we do not need this columny anymore for an lstm model

df.to_pickle('/content/drive/MyDrive/Python_data/data_processed_for_lstm.pkl')

with open("/content/drive/MyDrive/Python_data/chars_dict.json", "w") as outfile:
    json.dump(chars_dict, outfile)

In [ ]:
df.head()

,proj_id,language,clean
0,10001,JavaScript,"[/, /, , F, u, n, c, t, i, o, n, s, , a, s, ..."
1,10002,JavaScript,"[f, u, n, c, t, i, o, n, , m, e, d, i, a, n, ..."
2,10003,JavaScript,"[f, u, n, c, t, i, o, n, , d, i, v, B, y, Z, ..."
3,10004,JavaScript,"[i, f, , (, "", a, b, s, "", , i, n, , M, a, ..."
4,10005,JavaScript,"[(, f, u, n, c, t, i, o, n, , (, ), , {, \n,..."


Let's prepare data for the LSTM model.

In [ ]:
# we build a basic dictionary
char_to_int = dict((c, i) for i, c in enumerate(chars_dict.keys()))

# we will convert chars to integers
def code_to_int(code):
    return [char_to_int[x] for x in code]

df['data'] = df['clean'].apply(lambda x: code_to_int(x))
df['len'] = df['clean'].apply(len)
df.head(10)

,proj_id,language,clean,data,len
0,10001,JavaScript,"[/, /, , F, u, n, c, t, i, o, n, s, , a, s, ...","[54, 54, 96, 28, 80, 93, 81, 94, 91, 87, 93, 8...",905
1,10002,JavaScript,"[f, u, n, c, t, i, o, n, , m, e, d, i, a, n, ...","[77, 80, 93, 81, 94, 91, 87, 93, 96, 78, 95, 8...",1778
2,10003,JavaScript,"[f, u, n, c, t, i, o, n, , d, i, v, B, y, Z, ...","[77, 80, 93, 81, 94, 91, 87, 93, 96, 82, 91, 5...",332
3,10004,JavaScript,"[i, f, , (, "", a, b, s, "", , i, n, , M, a, ...","[91, 77, 96, 84, 74, 89, 66, 88, 74, 96, 91, 9...",70
4,10005,JavaScript,"[(, f, u, n, c, t, i, o, n, , (, ), , {, \n,...","[84, 77, 80, 93, 81, 94, 91, 87, 93, 96, 84, 8...",1614
5,10006,JavaScript,"[v, a, r, , _, , =, , [, \n, , , [, , 4,...","[59, 89, 90, 96, 55, 96, 76, 96, 62, 92, 96, 9...",887
6,10007,JavaScript,"[f, u, n, c, t, i, o, n, , s, h, e, l, l, S, ...","[77, 80, 93, 81, 94, 91, 87, 93, 96, 88, 73, 9...",478
7,10008,JavaScript,"[[, [, 1, ,, , 3, ,, , 5, ], ,, , [, 2, ,, ...","[62, 62, 70, 83, 96, 42, 83, 96, 37, 61, 83, 9...",1304
8,10009,JavaScript,"[(, f, u, n, c, t, i, o, n, , (, ), , {, \n,...","[84, 77, 80, 93, 81, 94, 91, 87, 93, 96, 84, 8...",1154
9,10010,JavaScript,"[(, f, u, n, c, t, i, o, n, , (, ), , {, \n,...","[84, 77, 80, 93, 81, 94, 91, 87, 93, 96, 84, 8...",3526


In [ ]:
df['len'].describe()

count    10006.000000
mean      1133.497801
std       1639.754254
min          3.000000
25%        268.250000
50%        651.000000
75%       1372.000000
max      40952.000000
Name: len, dtype: float64

We can see projects' lengths are very different - from just 3 chars to over 40 thousand. We'll have to make some decisions on padding the data for the model.

This notebook is getting a bit longer than I expected, so I will go quickly through removing excess white spaces in the code. In our char_to_int dictionary white space has value 96, we'll strip the code areound them. In some codes as much as 40% of chars were white spaces.

In [ ]:
def strip_spaces(line):
    return [x for i, x in enumerate(line) if (line[i - 1] != x or i == 0)]

df['data_stripped'] = df['data'].apply(strip_spaces)
df['len_stripped'] = df['data_stripped'].apply(len)

In [ ]:
df.head(10)

,proj_id,language,clean,data,len,data_stripped,len_stripped
0,10001,JavaScript,"[/, /, , F, u, n, c, t, i, o, n, s, , a, s, ...","[54, 54, 96, 28, 80, 93, 81, 94, 91, 87, 93, 8...",905,"[54, 96, 28, 80, 93, 81, 94, 91, 87, 93, 88, 9...",862
1,10002,JavaScript,"[f, u, n, c, t, i, o, n, , m, e, d, i, a, n, ...","[77, 80, 93, 81, 94, 91, 87, 93, 96, 78, 95, 8...",1778,"[77, 80, 93, 81, 94, 91, 87, 93, 96, 78, 95, 8...",1303
2,10003,JavaScript,"[f, u, n, c, t, i, o, n, , d, i, v, B, y, Z, ...","[77, 80, 93, 81, 94, 91, 87, 93, 96, 82, 91, 5...",332,"[77, 80, 93, 81, 94, 91, 87, 93, 96, 82, 91, 5...",312
3,10004,JavaScript,"[i, f, , (, "", a, b, s, "", , i, n, , M, a, ...","[91, 77, 96, 84, 74, 89, 66, 88, 74, 96, 91, 9...",70,"[91, 77, 96, 84, 74, 89, 66, 88, 74, 96, 91, 9...",63
4,10005,JavaScript,"[(, f, u, n, c, t, i, o, n, , (, ), , {, \n,...","[84, 77, 80, 93, 81, 94, 91, 87, 93, 96, 84, 8...",1614,"[84, 77, 80, 93, 81, 94, 91, 87, 93, 96, 84, 8...",1236
5,10006,JavaScript,"[v, a, r, , _, , =, , [, \n, , , [, , 4,...","[59, 89, 90, 96, 55, 96, 76, 96, 62, 92, 96, 9...",887,"[59, 89, 90, 96, 55, 96, 76, 96, 62, 92, 96, 6...",744
6,10007,JavaScript,"[f, u, n, c, t, i, o, n, , s, h, e, l, l, S, ...","[77, 80, 93, 81, 94, 91, 87, 93, 96, 88, 73, 9...",478,"[77, 80, 93, 81, 94, 91, 87, 93, 96, 88, 73, 9...",393
7,10008,JavaScript,"[[, [, 1, ,, , 3, ,, , 5, ], ,, , [, 2, ,, ...","[62, 62, 70, 83, 96, 42, 83, 96, 37, 61, 83, 9...",1304,"[62, 70, 83, 96, 42, 83, 96, 37, 61, 83, 96, 6...",1080
8,10009,JavaScript,"[(, f, u, n, c, t, i, o, n, , (, ), , {, \n,...","[84, 77, 80, 93, 81, 94, 91, 87, 93, 96, 84, 8...",1154,"[84, 77, 80, 93, 81, 94, 91, 87, 93, 96, 84, 8...",955
9,10010,JavaScript,"[(, f, u, n, c, t, i, o, n, , (, ), , {, \n,...","[84, 77, 80, 93, 81, 94, 91, 87, 93, 96, 84, 8...",3526,"[84, 77, 80, 93, 81, 94, 91, 87, 93, 96, 84, 8...",2581


In [ ]:
df['len_stripped'].max()

34805

The longest code was stripped from over 40k to around 35k.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import StratifiedKFold, train_test_split
import tensorflow as tf
import os
import json
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

Keeping in mind that source code lengths vary from 3 to almost 35.000 chars, we'll try padding it to 1000 chars. It's a very arbitrary choice.

We will also one-hot label our label column, as this is an LSTM model requirement for a categorical_crossentropy loss funcation in Keras.

In [ ]:
# preparing a dataset
max_len = 1000 # that is tough question
max_value = max(char_to_int.values()) # for normalizing values to a [0, 1] range
X = pad_sequences(df['data_stripped'], maxlen = max_len, dtype = 'float32') / max_value
X = X.reshape(X.shape[0], X.shape[1], 1) # reshaping for LSTM layer

one_hot = LabelBinarizer()
Y = one_hot.fit_transform(df['language'])

In [ ]:
pd.DataFrame(X.squeeze()).head(10) # this is just to show the data - Colab prints a DataFrame in a much more readable way than a numpy array

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.885417,0.875000,0.718750,0.781250,0.385417,0.885417,0.708333,0.958333,0.583333,0.958333
1,0.385417,0.958333,0.812500,0.989583,0.854167,0.947917,0.927083,0.968750,0.875000,0.645833,...,0.968750,0.885417,0.708333,0.958333,0.583333,0.885417,0.875000,0.885417,0.708333,0.958333
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.989583,0.937500,0.906250,0.875000,0.718750,0.864583,0.718750,0.885417,0.708333,0.958333
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.854167,0.770833,0.885417,1.000000,0.593750,1.000000,0.781250,1.000000,0.583333,0.958333
4,0.916667,0.645833,0.947917,0.635417,0.708333,0.958333,1.000000,0.583333,0.885417,0.958333,...,0.635417,0.885417,0.708333,0.958333,0.583333,0.885417,0.875000,0.885417,0.708333,0.958333
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.979167,0.833333,0.937500,0.968750,1.000000,0.406250,0.708333,0.958333,0.583333,0.958333
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.906250,0.947917,0.968750,0.875000,0.770833,1.000000,0.770833,0.885417,0.708333,0.958333
7,0.989583,0.979167,0.833333,0.937500,0.968750,1.000000,0.895833,0.916667,0.979167,0.645833,...,0.635417,0.885417,0.708333,0.958333,0.583333,0.885417,0.875000,0.885417,0.708333,0.958333
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.885417,0.708333,0.958333,0.583333,0.885417,0.875000,0.885417,0.708333,0.958333
9,0.989583,0.927083,0.916667,1.000000,0.947917,0.968750,0.979167,0.989583,0.750000,0.989583,...,0.718750,0.218750,0.385417,0.729167,0.312500,0.729167,0.312500,0.541667,0.885417,0.958333


In [ ]:
Y[:10]

array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

Let's build a model. Two bidirectinal LSTM layers with 64 cells in each layer, and dropout in between.

In [ ]:
def get_model():
    model = Sequential()
    model.add(Bidirectional(LSTM(64, return_sequences=True, input_shape=(X.shape[1], 1))))
    # model.add(Bidirectional(LSTM(100, input_shape=(X.shape[1], 1))))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dropout(0.5))
    model.add(Dense(Y.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model = get_model()

This is just a task, so we will not run a StratifiedKFold, as we should do in a real world. For the sake of simplicity, we'll use a simple train_test_split.

If a dataset was larger, we could also keep a validation set for a final check of the model.

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, train_size = 0.8, shuffle = True, stratify = Y)

Untill now, I haven't touched a question of metrics. We have a multilabel classification with a rather unballanced dataset. In a real world, we could have some discussion on whether accuracy should be the only metrics to be applied. But again, for the sake of simplicity, we'll stick with accuracy. After all, analyzing recall / precision metrics or confusion matrix for 19 classes may get a bit overwhelming. And we've all seen even more unballanced datasets. Neither we will address this challenge with any other method, like data augmentation etc.

The other obvious things that we'll drop now are hyperparameter optimization, testing different architectures, optimizers, using callbacks etc. We'll just fit it as it is for 100 epochs and we'll see.

And last but not least - I assumed using an Embedding layer before LSTM for a source code makes a limited sense. Programming language is not a natural language and "words" don't have a "meaning" as we understand it. For an NLP task adding an Embedding layer would definitely add value.

In [ ]:
history = model.fit(xtrain, ytrain, validation_data = (xtest, ytest), epochs = 100, batch_size = 64, verbose = 2)

Epoch 1/100
126/126 - 31s - loss: 2.8852 - accuracy: 0.0831 - val_loss: 2.8767 - val_accuracy: 0.0894 - 31s/epoch - 249ms/step
Epoch 2/100
126/126 - 22s - loss: 2.8660 - accuracy: 0.0908 - val_loss: 2.8714 - val_accuracy: 0.0929 - 22s/epoch - 174ms/step
Epoch 3/100
126/126 - 22s - loss: 2.8675 - accuracy: 0.0896 - val_loss: 2.8714 - val_accuracy: 0.0894 - 22s/epoch - 174ms/step
Epoch 4/100
126/126 - 22s - loss: 2.8617 - accuracy: 0.0953 - val_loss: 2.8684 - val_accuracy: 0.1039 - 22s/epoch - 172ms/step
Epoch 5/100
126/126 - 22s - loss: 2.8577 - accuracy: 0.0942 - val_loss: 2.8736 - val_accuracy: 0.0779 - 22s/epoch - 174ms/step
Epoch 6/100
126/126 - 22s - loss: 2.8537 - accuracy: 0.0951 - val_loss: 2.8605 - val_accuracy: 0.1064 - 22s/epoch - 173ms/step
Epoch 7/100
126/126 - 22s - loss: 2.8426 - accuracy: 0.1041 - val_loss: 2.8245 - val_accuracy: 0.1169 - 22s/epoch - 174ms/step
Epoch 8/100
126/126 - 22s - loss: 2.7938 - accuracy: 0.1261 - val_loss: 2.7693 - val_accuracy: 0.1354 - 22s/epo

After 100 epochs we reached 60% of validation accuracy. Not that good as we expected, but LSTM networks have a tendency to excel after a long train period.

So let's give it a chance for another 100 epochs with the same pretrained model.

In [ ]:
history2 = model.fit(xtrain, ytrain, validation_data = (xtest, ytest), epochs = 100, batch_size = 64, verbose = 2)

Epoch 1/100
126/126 - 22s - loss: 1.3900 - accuracy: 0.5706 - val_loss: 1.3155 - val_accuracy: 0.6069 - 22s/epoch - 177ms/step
Epoch 2/100
126/126 - 23s - loss: 1.3555 - accuracy: 0.5803 - val_loss: 1.3268 - val_accuracy: 0.6079 - 23s/epoch - 183ms/step
Epoch 3/100
126/126 - 22s - loss: 1.3939 - accuracy: 0.5687 - val_loss: 1.3010 - val_accuracy: 0.6014 - 22s/epoch - 176ms/step
Epoch 4/100
126/126 - 23s - loss: 1.3433 - accuracy: 0.5832 - val_loss: 1.3240 - val_accuracy: 0.6084 - 23s/epoch - 181ms/step
Epoch 5/100
126/126 - 22s - loss: 1.3282 - accuracy: 0.5893 - val_loss: 1.3141 - val_accuracy: 0.5934 - 22s/epoch - 174ms/step
Epoch 6/100
126/126 - 22s - loss: 1.3067 - accuracy: 0.5938 - val_loss: 1.3308 - val_accuracy: 0.5944 - 22s/epoch - 174ms/step
Epoch 7/100
126/126 - 22s - loss: 1.3164 - accuracy: 0.5927 - val_loss: 1.2725 - val_accuracy: 0.6154 - 22s/epoch - 174ms/step
Epoch 8/100
126/126 - 22s - loss: 1.3001 - accuracy: 0.6053 - val_loss: 1.2967 - val_accuracy: 0.6074 - 22s/epo

Still rising, so let's try another 100 epochs.

In [ ]:
history3 = model.fit(xtrain, ytrain, validation_data = (xtest, ytest), epochs = 100, batch_size = 64, verbose = 2)

Epoch 1/100
126/126 - 23s - loss: 0.8405 - accuracy: 0.7284 - val_loss: 1.0580 - val_accuracy: 0.6958 - 23s/epoch - 182ms/step
Epoch 2/100
126/126 - 22s - loss: 0.8410 - accuracy: 0.7255 - val_loss: 1.0620 - val_accuracy: 0.6963 - 22s/epoch - 176ms/step
Epoch 3/100
126/126 - 22s - loss: 0.8610 - accuracy: 0.7220 - val_loss: 1.0610 - val_accuracy: 0.6983 - 22s/epoch - 174ms/step
Epoch 4/100
126/126 - 22s - loss: 0.8449 - accuracy: 0.7310 - val_loss: 1.1049 - val_accuracy: 0.6793 - 22s/epoch - 173ms/step
Epoch 5/100
126/126 - 22s - loss: 0.8378 - accuracy: 0.7351 - val_loss: 1.0719 - val_accuracy: 0.6948 - 22s/epoch - 175ms/step
Epoch 6/100
126/126 - 22s - loss: 0.8371 - accuracy: 0.7305 - val_loss: 1.0781 - val_accuracy: 0.6913 - 22s/epoch - 174ms/step
Epoch 7/100
126/126 - 22s - loss: 0.8100 - accuracy: 0.7389 - val_loss: 1.1168 - val_accuracy: 0.6758 - 22s/epoch - 174ms/step
Epoch 8/100
126/126 - 22s - loss: 0.8062 - accuracy: 0.7451 - val_loss: 1.0695 - val_accuracy: 0.6883 - 22s/epo

That's for it. We stuck around 71% of validation accuracy and we see that the model has probably begun to overfit a little bit.

Conclusion: it's not the first and not the last time when a simpler model from a sklearn toolbox resulted in much better performance than a deep learning model.